In [9]:
import os
from pathlib import Path
from dotenv import load_dotenv

## Loading Environment File

In [10]:
# Try loading from environment first, then from dev.env file
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if not GOOGLE_API_KEY:
    secret_path = Path("../.env")
    print("Looking for dev.env at:", secret_path.resolve())
    if secret_path.exists():
        load_dotenv(secret_path)
        GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
    
    if not GOOGLE_API_KEY:
        raise ValueError("GOOGLE_API_KEY not found in environment variables or dev.env file.")

print("GOOGLE_API_KEY loaded:", bool(GOOGLE_API_KEY))

GOOGLE_API_KEY loaded: True


## Setup: Imports, Retry Config

In [11]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [12]:
import sys

# Ensure the project root (the parent of the "src" directory) is on sys.path
# so that "import src.model" finds the src package under the project root.
project_root = Path.cwd().parent
src_dir = project_root / "src"

project_root_path = str(project_root.resolve())
if project_root_path not in sys.path:
    sys.path.insert(0, project_root_path)

from src.model import Intensity, SentimentOutput
from src.model.rag_output import RagOutput, SuccessResponse, ErrorResponse

In [13]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

## Agents Creation

In [14]:
rag_output = RagOutput(
    result=SuccessResponse(
        response="This is a sample response from the RAG output model."
    )
)
sentiment_output = SentimentOutput(
    frustration_level=Intensity.LOW,
    neutrality_level=Intensity.LOW,
    satisfaction_level=Intensity.HIGH,
    confusion_level=Intensity.MEDIUM
)

In [15]:
# The AggregatorAgent runs *after* the parallel step to synthesize the results.
aggregator_agent = Agent(
    name="AggregatorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # It uses placeholders to inject the outputs from the parallel agents, which are now in the session state.
    instruction=f"""Combine these findings into a single executive summary:

    **RAG output:**
    {rag_output}
    
    **User sentiment analysis:**
    {sentiment_output}
""",
    output_key="final_output",  # This will be the final output of the entire system.
)

print("✅ aggregator_agent created.")

✅ aggregator_agent created.


## Engine Def

Check the instructions:

In [18]:
aggregator_agent.instruction

"Combine these findings into a single executive summary:\n\n    **RAG output:**\n    result=SuccessResponse(status='success', response='This is a sample response from the RAG output model.')\n\n    **User sentiment analysis:**\n    frustration_level='low' confusion_level='medium' satisfaction_level='high' neutrality_level='low'\n"

In [16]:
runner = InMemoryRunner(agent=aggregator_agent)

## Testing

In [17]:
response = await runner.run_debug(
    "Should I vaccinate?"
)


 ### Created new session: debug_session_id

User > Should I vaccinate?
AggregatorAgent > Vaccination is a safe and effective way to protect yourself and your community from serious diseases. Major health organizations worldwide, including the Centers for Disease Control and Prevention (CDC) and the World Health Organization (WHO), recommend vaccination as a crucial public health measure.

Here's why vaccination is generally recommended:

*   **Disease Prevention:** Vaccines significantly reduce your risk of contracting preventable diseases like measles, polio, influenza, and COVID-19. These diseases can lead to severe illness, hospitalization, long-term health complications, and even death.
*   **Community Protection (Herd Immunity):** When a high percentage of the population is vaccinated, it becomes much harder for diseases to spread. This protects those who cannot be vaccinated, such as infants, the elderly, and individuals with weakened immune systems.
*   **Safety and Effectivene